In [1]:
import re
import pandas as pd

## **0.** Limpieza y armado del dataset

In [2]:
whr = pd.read_excel("../datasets/WHR25_Data_Figure_2.1v3.xlsx")
qog_ts = pd.read_csv('../datasets/qog_bas_ts_jan25.csv')

In [3]:
qog = qog_ts.query("year >= 2011")[
    [
        "cname",
        "year",
        "ti_cpi",
        "vdem_polyarchy",
        "wdi_gini",
        "undp_hdi",
        "wdi_unempilo",
        "wdi_lifexp",
        "wdi_internet"
    ]
].copy()

In [4]:
country_mapping = {
    "Bolivia": 'Bolivia (Plurinational State of)',
    "Central African Republic": 'Central African Republic (the)',
    "Comoros": 'Comoros (the)',
    "Congo": 'Congo (the)',
    "Côte d’Ivoire": "Côte d'Ivoire",
    "Dominican Republic": 'Dominican Republic (the)',
    "DR Congo": 'Congo (the Democratic Republic of the)',
    "Gambia": 'Gambia (the)',
    "Iran": 'Iran (Islamic Republic of)',
    "Lao PDR": "Lao People's Democratic Republic (the)",
    "Netherlands": 'Netherlands (the)',
    "Niger": 'Niger (the)',
    "Philippines": 'Philippines (the)',
    "Republic of Korea": 'Korea (the Republic of)',
    "Republic of Moldova": 'Moldova (the Republic of)',
    "Russian Federation": 'Russian Federation (the)',
    "Sudan": 'Sudan (the)',
    "Syria": 'Syrian Arab Republic (the)',
    "Tanzania": 'Tanzania, the United Republic of',
    "Türkiye": 'Turkey',
    "United Arab Emirates": 'United Arab Emirates (the)',
    "United Kingdom": 'United Kingdom of Great Britain and Northern Ireland (the)',
    "United States": 'United States of America (the)',
    "Venezuela": 'Venezuela (Bolivarian Republic of)',
    'Swaziland': 'Eswatini'
}
whr['Country name'] = whr['Country name'].replace(country_mapping)
whr_countries = whr['Country name'].unique()
qog_countries = qog.cname.unique()

countries_to_drop = [c for c in whr_countries if c not in qog_countries]
whr_clean = whr[~whr['Country name'].isin(countries_to_drop)].copy()

In [5]:
df = whr_clean.merge(
    qog, left_on=["Country name", "Year"], right_on=["cname", "year"], how="left"
).drop(["cname", "year"], axis=1)

df = df.rename(
    columns={
        "ti_cpi": "corruption_index",
        "vdem_polyarchy": "democracy_index",
        "wdi_gini": "gini_coefficient",
        "wdi_internet": "internet_pct",
        "undp_hdi": "human_dev_index",
        "wdi_unempilo": "unemployment_pct",
        "wdi_lifexp": "life_expectancy",
        "wdi_popurb": "internet_access_pct",
    }
)

In [6]:
prefijo_patron = r"^Explained by:\s*"
sufijo_patron = r"\s*\(3-year average\)$"

col_names = []
for c in df.columns:
    _name = re.sub(prefijo_patron, "", c)
    _name = re.sub(sufijo_patron, "", _name)
    _name = _name.replace(" ", "_").replace("+", "").replace("__", "_")
    col_names.append(_name.lower())

df.columns = col_names
df_clean = df.query("year >= 2014 and year <= 2023").copy()

In [7]:
# Mapeo de países a continentes
country_to_continent = {
    # África
    "Algeria": "Africa",
    "Angola": "Africa",
    "Benin": "Africa",
    "Botswana": "Africa",
    "Burkina Faso": "Africa",
    "Burundi": "Africa",
    "Cameroon": "Africa",
    "Central African Republic (the)": "Africa",
    "Chad": "Africa",
    "Comoros (the)": "Africa",
    "Congo (the)": "Africa",
    "Congo (the Democratic Republic of the)": "Africa",
    "Djibouti": "Africa",
    "Egypt": "Africa",
    "Eswatini": "Africa",
    "Ethiopia": "Africa",
    "Gabon": "Africa",
    "Gambia (the)": "Africa",
    "Ghana": "Africa",
    "Guinea": "Africa",
    "Kenya": "Africa",
    "Lesotho": "Africa",
    "Liberia": "Africa",
    "Libya": "Africa",
    "Madagascar": "Africa",
    "Malawi": "Africa",
    "Mali": "Africa",
    "Mauritania": "Africa",
    "Mauritius": "Africa",
    "Morocco": "Africa",
    "Mozambique": "Africa",
    "Namibia": "Africa",
    "Niger (the)": "Africa",
    "Nigeria": "Africa",
    "Rwanda": "Africa",
    "Senegal": "Africa",
    "Sierra Leone": "Africa",
    "Somalia": "Africa",
    "South Africa": "Africa",
    "South Sudan": "Africa",
    "Sudan (the)": "Africa",
    "Tanzania, the United Republic of": "Africa",
    "Togo": "Africa",
    "Tunisia": "Africa",
    "Uganda": "Africa",
    "Zambia": "Africa",
    "Zimbabwe": "Africa",
    # Asia
    "Afghanistan": "Asia",
    "Armenia": "Asia",
    "Azerbaijan": "Asia",
    "Bahrain": "Asia",
    "Bangladesh": "Asia",
    "Bhutan": "Asia",
    "Cambodia": "Asia",
    "China": "Asia",
    "Georgia": "Asia",
    "India": "Asia",
    "Indonesia": "Asia",
    "Iran (Islamic Republic of)": "Asia",
    "Iraq": "Asia",
    "Israel": "Asia",
    "Japan": "Asia",
    "Jordan": "Asia",
    "Kazakhstan": "Asia",
    "Kuwait": "Asia",
    "Kyrgyzstan": "Asia",
    "Lao People's Democratic Republic (the)": "Asia",
    "Lebanon": "Asia",
    "Malaysia": "Asia",
    "Maldives": "Asia",
    "Mongolia": "Asia",
    "Myanmar": "Asia",
    "Nepal": "Asia",
    "Oman": "Asia",
    "Pakistan": "Asia",
    "Philippines (the)": "Asia",
    "Qatar": "Asia",
    "Korea (the Republic of)": "Asia",
    "Saudi Arabia": "Asia",
    "Singapore": "Asia",
    "Sri Lanka": "Asia",
    "Syrian Arab Republic (the)": "Asia",
    "Tajikistan": "Asia",
    "Thailand": "Asia",
    "Turkey": "Asia",
    "Turkmenistan": "Asia",
    "United Arab Emirates (the)": "Asia",
    "Uzbekistan": "Asia",
    "Viet Nam": "Asia",
    "Yemen": "Asia",
    # Europa
    "Albania": "Europe",
    "Austria": "Europe",
    "Belarus": "Europe",
    "Belgium": "Europe",
    "Bosnia and Herzegovina": "Europe",
    "Bulgaria": "Europe",
    "Croatia": "Europe",
    "Cyprus": "Europe",
    "Czechia": "Europe",
    "Denmark": "Europe",
    "Estonia": "Europe",
    "Finland": "Europe",
    "France": "Europe",
    "Germany": "Europe",
    "Greece": "Europe",
    "Hungary": "Europe",
    "Iceland": "Europe",
    "Ireland": "Europe",
    "Italy": "Europe",
    "Latvia": "Europe",
    "Lithuania": "Europe",
    "Luxembourg": "Europe",
    "Malta": "Europe",
    "Moldova (the Republic of)": "Europe",
    "Montenegro": "Europe",
    "Netherlands (the)": "Europe",
    "North Macedonia": "Europe",
    "Norway": "Europe",
    "Poland": "Europe",
    "Portugal": "Europe",
    "Romania": "Europe",
    "Russian Federation (the)": "Europe",
    "Serbia": "Europe",
    "Slovakia": "Europe",
    "Slovenia": "Europe",
    "Spain": "Europe",
    "Sweden": "Europe",
    "Switzerland": "Europe",
    "Ukraine": "Europe",
    "United Kingdom of Great Britain and Northern Ireland (the)": "Europe",
    # América del Norte
    "Belize": "North America",
    "Canada": "North America",
    "Costa Rica": "North America",
    "Cuba": "North America",
    "Dominican Republic (the)": "North America",
    "El Salvador": "North America",
    "Guatemala": "North America",
    "Haiti": "North America",
    "Honduras": "North America",
    "Jamaica": "North America",
    "Mexico": "North America",
    "Nicaragua": "North America",
    "Panama": "North America",
    "Trinidad and Tobago": "North America",
    "United States of America (the)": "North America",
    # América del Sur
    "Argentina": "South America",
    "Bolivia (Plurinational State of)": "South America",
    "Brazil": "South America",
    "Chile": "South America",
    "Colombia": "South America",
    "Ecuador": "South America",
    "Guyana": "South America",
    "Paraguay": "South America",
    "Peru": "South America",
    "Suriname": "South America",
    "Uruguay": "South America",
    "Venezuela (Bolivarian Republic of)": "South America",
    # Oceanía
    "Australia": "Oceania",
    "New Zealand": "Oceania",
}

# Agregar la columna de continente
df_clean["region"] = df_clean["country_name"].map(country_to_continent)

# Verificar si hay países sin continente asignado
missing_continents = df_clean[df_clean["region"].isna()]["country_name"].unique()
if len(missing_continents) > 0:
    print("Países sin continente asignado:")
    print(missing_continents)
else:
    print("✓ Todos los países tienen continente asignado")

print(f"\nDistribución por continente:")
print(df_clean["region"].value_counts().sort_index())

✓ Todos los países tienen continente asignado

Distribución por continente:
region
Africa           411
Asia             392
Europe           398
North America    126
Oceania           20
South America    102
Name: count, dtype: int64


In [8]:
df_clean.to_csv("../datasets/whr+qog.csv", index=False)